In [1]:
from datetime import datetime, timedelta
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup

In [19]:
def jobTypeFormatter(_type):
    if _type == "fulltime":
        return "Full-Time"
    elif _type == "parttime":
        return "Part-Time"
    elif _type == "contract":
        return "Contract"
    else:
        return "Internship"

def jobSearchFormatter(search):
    return " ".join([i.capitalize() for i in search.split("-")])

def scrape_raw_glassdoor():
    apiKey = "9edb1daaf83df2801254221d6202b156"
    # apiKey="cfd0d3475f3d2cd5a9f6e13f9189c52a"
    def remove_non_numbers(s):
        return re.sub(r'\D+', '', s)

    def convert_pay_to_range(pay_string):
        whatWeNeed = pay_string.split()[0]
        if "−" in whatWeNeed:
            ranges = [float(remove_non_numbers(i)) * 1000 for i in whatWeNeed.split("−")]
            return ranges
        else:
            return [float(i) * 1000 for i in [remove_non_numbers(whatWeNeed), remove_non_numbers(whatWeNeed)]]
    
    types = [
        "fulltime",
        "parttime",
        "contract",
        "internship"
    ]
    
    seniorityTypes = [
        "internship",
        "entrylevel",
        "midseniorlevel",
        "director",
        "executive"
    ]
    
    jobSearches = [
        "software-engineer",
        "data-analyst",
        "data-engineer",
        "data-scientist",
        "product-manager",
        "systems-analyst",
        "it-security-analyst",
        "devops-engineer",
        "cloud-architect",
        "app-developer",
        "web-developer"
    ]
    counter = 1
    page_counter = 1
    jobs = []
    for search in jobSearches:
        print("Beginning search ", counter)
        counter += 1
        for seniority in seniorityTypes:
            for _type in types:
                url = f"https://www.glassdoor.sg/Job/{search}-jobs-SRCH_KO0,{len(search)}.htm?jobType={_type}?fromAge=30&seniorityType={seniority}"
                # print(url)
                payload = {'api_key': apiKey, 'url': url}
                r = requests.get('http://api.scraperapi.com', params=payload)
                soup = BeautifulSoup(r.text.encode('utf-8'), "html.parser")
                job_listings = soup.find_all('li', {'class': lambda x: x and x.startswith('react-job-listing')})
                # print(len(job_listings))
                for job in job_listings:
                    companyName = job.find(class_="css-l2wjgv e1n63ojh0 jobLink").text
                    jobTitle = job.find(class_="jobLink css-1rd3saf eigr9kq2").text
                    location = "Singapore"
                    #location = job.find(class_="d-flex flex-wrap css-11d3uq0 e1rrn5ka2").text
                    pay = job.find(class_="css-1xe2xww e1wijj242")
                    salaryFrequency = "Monthly"
                    if pay:
                        pay = pay.text
                        salaryLowerbound, salaryUpperbound = convert_pay_to_range(pay)
                        if salaryLowerbound > 20000 or salaryUpperbound > 20000:
                            salaryFrequency = "Yearly"
                    else:
                        pay = "Not disclosed"
                        continue
                    jobs.append([jobTitle, search, companyName, salaryLowerbound, salaryUpperbound, salaryFrequency,
                                 _type, "Glassdoor", url])
                    # print("Appended {}".format(page_counter))
                    page_counter += 1
                    time.sleep(2)
                    # print(jobs)
        #         break
        #     break
        # break
    # print("LENGTH OF JOBS:",len(jobs))
    header = ["Job Title", "Searched Job Title", "Company Name", "Salary Lower Bound", "Salary Upper Bound", "Salary Frequency", "Job Type", "Website", "URL"]               
    df = pd.DataFrame(jobs, columns = header) 
    return df

In [15]:
def data_processing(df):
    df['Searched Job Title'] = df['Searched Job Title'].apply(jobSearchFormatter)
    df['Job Type'] = df['Job Type'].apply(jobTypeFormatter)
    return df

In [16]:
df = scrape_raw_glassdoor()

Beginning search  1


In [17]:
new_df = data_processing(df)

In [18]:
new_df.head(5)

,Job Title,Searched Job Title,Company Name,Salary Lower Bound,Salary Upper Bound,Salary Frequency,Job Type,Website,URL
0,Software Engineer Intern,Software Engineer,Alpha Impact,1000.0,1000.0,Monthly,Full-Time,Glassdoor,https://www.glassdoor.sg/Job/software-engineer...
1,Fullstack Software Engineer Intern,Software Engineer,HASHSTACS PTE. LTD.,1000.0,1000.0,Monthly,Full-Time,Glassdoor,https://www.glassdoor.sg/Job/software-engineer...
2,Full Stack Engineer Intern (Fall 2023),Software Engineer,Nika.eco,1000.0,1000.0,Monthly,Full-Time,Glassdoor,https://www.glassdoor.sg/Job/software-engineer...
3,Software Developer Intern,Software Engineer,FoodLine,2000.0,2000.0,Monthly,Full-Time,Glassdoor,https://www.glassdoor.sg/Job/software-engineer...
4,Software Engineering Intern,Software Engineer,Kabam Robotics,1000.0,1000.0,Monthly,Full-Time,Glassdoor,https://www.glassdoor.sg/Job/software-engineer...
